## [--- Day 12: Secret Entrance ---](https://adventofcode.com/2025/day/12)

In [ ]:
import enum
import re
from dataclasses import dataclass


@dataclass
class Grid:
    width: int
    height: int
    grid: int
    requirements: tuple[int, int, int, int, int, int]


def build_grids(data: list[re.Match]) -> list[Grid]:
    grids: list[Grid] = []
    for _, dimensions, reqs in data:
        shape_requirements = tuple(map(int, reqs.split()))
        if len(shape_requirements) != 6:
            raise ValueError("Each requirement must have exactly 6 integers.")
        x, y = map(int, dimensions.split("x"))
        grids.append(
            Grid(width=x, height=y, grid=x * y, requirements=shape_requirements[0:6])
        )

    return grids


def build_shapes(raw_shapes: list[re.Match]) -> list[set[frozenset[tuple[int, int]]]]:
    all_shapes: list[set[frozenset[tuple[int, int]]]] = []
    for match in raw_shapes:
        raw_shape = parse_shape(match[0].splitlines())
        shape_set: set[frozenset[tuple[int, int]]] = set([raw_shape])

        # Generate all rotations and reflections
        for _ in range(3):
            raw_shape = rotate(raw_shape)
            shape_set.add(raw_shape)
            shape_set.add(reflect(raw_shape))

        all_shapes.append(set(shape_set))

    return all_shapes


def parse_shape(raw_shape: list[str]) -> frozenset[tuple[int, int]]:
    coords: list[tuple[int, int]] = [
        (x, y)
        for y, row in enumerate(raw_shape)
        for x, char in enumerate(row)
        if char == "#"
    ]
    return frozenset(coords)


def normalise(shape: list[tuple[int, int]]) -> list[tuple[int, int]]:
    min_x = min([x for x, _ in shape])
    min_y = min([y for _, y in shape])
    return [(x - min_x, y - min_y) for x, y in shape]


def rotate(shape: frozenset[tuple[int, int]]) -> frozenset[tuple[int, int]]:
    return frozenset(normalise([(-y, x) for x, y in shape]))


def reflect(shape: frozenset[tuple[int, int]]) -> frozenset[tuple[int, int]]:
    return frozenset(normalise([(-x, y) for x, y in shape]))


def print_shape(shape: frozenset[tuple[int, int]]) -> None:
    max_x = max(x for x, _ in shape)
    max_y = max(y for _, y in shape)
    grid: list[list[str]] = [
        ["# " if (x, y) in shape else ". " for x in range(max_x + 1)]
        for y in range(max_y + 1)
    ]
    print("\n".join("".join(row) for row in grid))
    print("-" * (max_x * 2 + 1))


def solve() -> None:
    with open("..\\data\\12 example.txt") as file:
        pattern = re.compile(r"^([#.\n]{11})|(\d+x\d+): ((?:\d+ ?){6})$", re.MULTILINE)
        data = pattern.findall(file.read())

    raw_shapes = data[:6]
    shapes: list[set[frozenset[tuple[int, int]]]] = build_shapes(raw_shapes)
    for n, shape in enumerate(shapes, start=1):
        print(f"Shape {n}:")
        for variant in shape:
            print_shape(variant)

    grids: list[Grid] = build_grids(data[6:])

    results: list[tuple[Grid, bool]] = []


solve()

Shape 1:
# # . 
# # . 
# # # 
-----
# . . 
# # # 
# # # 
-----
# # # 
# # # 
. . # 
-----
. # # 
. # # 
# # # 
-----
. . # 
# # # 
# # # 
-----
# # # 
# # . 
# # . 
-----
# # # 
# # # 
# . . 
-----
Shape 2:
. # # 
# # . 
# # # 
-----
# # # 
# # . 
. # # 
-----
. # # 
# # # 
# . # 
-----
# # . 
# # # 
# . # 
-----
# # . 
. # # 
# # # 
-----
# . # 
# # # 
. # # 
-----
# . # 
# # # 
# # . 
-----
Shape 3:
# # . 
# # # 
. # # 
-----
. # # 
# # # 
# # . 
-----
Shape 4:
. # # 
# # # 
. # # 
-----
# # # 
# # # 
. # . 
-----
# # . 
# # # 
# # . 
-----
. # . 
# # # 
# # # 
-----
Shape 5:
# # # 
. . # 
# # # 
-----
# # # 
# . . 
# # # 
-----
# # # 
# . # 
# . # 
-----
# . # 
# . # 
# # # 
-----
Shape 6:
# # # 
. # . 
# # # 
-----
# . # 
# # # 
# . # 
-----


In [ ]:
shape1: list[str] = ["###", "##.", "##."]
shape1_v = [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (0, 2), (1, 2)]
shape1_r = reflect(rotate(shape1_v))
reconstructed = [["."] * 3 for _ in range(3)]
for x, y in shape1_r:
    reconstructed[y][x] = "#"


shape1_expected = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1)]
expected = [["."] * 3 for _ in range(3)]
for x, y in shape1_expected:
    expected[y][x] = "#"

print("start:")
print("\n".join(shape1))
print("rotated:")
print("\n".join("".join(row) for row in reconstructed))
print("expected:")
print("\n".join("".join(row) for row in expected))


start:
###
##.
##.
rotated:
###
###
#..
expected:
###
###
#..


I'm working on AoC-2025 problems to develop my programming skills. Here is the problem I am currently working on:

--- Day 12: Christmas Tree Farm ---
You're almost out of time, but there can't be much left to decorate. Although there are no stairs, elevators, escalators, tunnels, chutes, teleporters, firepoles, or conduits here that would take you deeper into the North Pole base, there is a ventilation duct. You jump in.
After bumping around for a few minutes, you emerge into a large, well-lit cavern full of Christmas trees!
There are a few Elves here frantically decorating before the deadline. They think they'll be able to finish most of the work, but the one thing they're worried about is the presents for all the young Elves that live here at the North Pole. It's an ancient tradition to put the presents under the trees, but the Elves are worried they won't fit.
The presents come in a few standard but very weird shapes. The shapes and the regions into which they need to fit are all measured in standard units. To be aesthetically pleasing, the presents need to be placed into the regions in a way that follows a standardized two-dimensional unit grid; you also can't stack presents.
As always, the Elves have a summary of the situation (your puzzle input) for you. First, it contains a list of the presents' shapes. Second, it contains the size of the region under each tree and a list of the number of presents of each shape that need to fit into that region. For example:

0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
The first section lists the standard present shapes. For convenience, each shape starts with its index and a colon; then, the shape is displayed visually, where # is part of the shape and . is not.
The second section lists the regions under the trees. Each line starts with the width and length of the region; 12x5 means the region is 12 units wide and 5 units long. The rest of the line describes the presents that need to fit into that region by listing the quantity of each shape of present; 1 0 1 0 3 2 means you need to fit one present with shape index 0, no presents with shape index 1, one present with shape index 2, no presents with shape index 3, three presents with shape index 4, and two presents with shape index 5.
Presents can be rotated and flipped as necessary to make them fit in the available space, but they have to always be placed perfectly on the grid. Shapes can't overlap (that is, the # part from two different presents can't go in the same place on the grid), but they can fit together (that is, the . part in a present's shape's diagram does not block another present from occupying that space on the grid).
The Elves need to know how many of the regions can fit the presents listed. In the above example, there are six unique present shapes and three regions that need checking.
The first region is 4x4:

....
....
....
....
In it, you need to determine whether you could fit two presents that have shape index 4:

###
#..
###
After some experimentation, it turns out that you can fit both presents in this region. Here is one way to do it, using A to represent one present and B to represent the other:

AAA.
ABAB
ABAB
.BBB
The second region, 12x5: 1 0 1 0 2 2, is 12 units wide and 5 units long. In that region, you need to try to fit one present with shape index 0, one present with shape index 2, two presents with shape index 4, and two presents with shape index 5.
It turns out that these presents can all fit in this region. Here is one way to do it, again using different capital letters to represent all the required presents:

....AAAFFE.E
.BBBAAFFFEEE
DDDBAAFFCECE
DBBB....CCC.
DDD.....C.C.
The third region, 12x5: 1 0 1 0 3 2, is the same size as the previous region; the only difference is that this region needs to fit one additional present with shape index 4. Unfortunately, no matter how hard you try, there is no way to fit all of the presents into this region.
So, in this example, 2 regions can fit all of their listed presents.
Consider the regions beneath each tree and the presents the Elves would like to fit into each of them. How many of the regions can fit all of the presents listed?

I've never tried to solve a problem like this before. I want to optimize for execution time and maintain best practices. The grid sizes approach 50x50, there are six 3x3 shapes, similar to the example.
I'm considering:
1. Use integers to represent the grid and where shapes could sit so I can use bitmasking.
2. create sets of each shape's rotations and reflections.
4. convert each grid into an integer.
3. convert shape coordinates into grid offsets.
4. calculate shape placements for each grid's dimensions.
5. recursively search each grid and backtrack to prune the search space.
   1. Create list of placements, per shape, as an int for this grid, where the index is the anchor bit.
   2. start with the most complex shape.
   3. bitwise floodfill to find the size of holes.
   4. skip holes that are too small for a given shape.
   5. exit if the remaining space is less than the minimum space required for all remaining shapes.

Please guide me through solving this problem in Python, but not provide any code unless I ask for it. Let's start with how to fix the rotation method to rotate the shape around it's centre instead of the origin. 